# Data Cleaning

### Imports

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import re

### Cleaning

In [20]:
# read in saved listings + reviews datasets from data collection notebook
listings_df = pd.read_csv('../data/listings.csv').drop(columns = 'Unnamed: 0')
reviews_df = pd.read_csv('../data/reviews.csv').drop(columns = 'Unnamed: 0')

/var/folders/bl/639jfgnd3jb_3b76z0p4htl40000gn/T/ipykernel_15827/2279010532.py:2: DtypeWarning: Columns (0,1,3,10,36,45,46,55,69) have mixed types. Specify dtype option on import or set low_memory=False.
  listings_df = pd.read_csv('../data/listings.csv').drop(columns = 'Unnamed: 0')
/var/folders/bl/639jfgnd3jb_3b76z0p4htl40000gn/T/ipykernel_15827/2279010532.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews_df = pd.read_csv('../data/reviews.csv').drop(columns = 'Unnamed: 0')


#### Listings DF

In [21]:
# check first
listings_df.head(1)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595.0,https://www.airbnb.com/rooms/2595,20221204162430.0,2022-12-05,city scrape,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,...,4.8,4.81,4.4,NaN,f,3.0,3.0,0.0,0.0,0.31


In [22]:
# check last
listings_df.tail(1)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
41534,772716724205003579,https://www.airbnb.com/rooms/772716724205003579,20221204162430,2022-12-05,city scrape,2 bedroom Condo near West Village,This beautifully decorated condo will give you...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,481177884,...,NaN,NaN,NaN,NaN,t,2.0,2.0,0.0,0.0,NaN


In [23]:
listings_df.dtypes

id                                               object
listing_url                                      object
scrape_id                                        object
last_scraped                                     object
source                                           object
                                                 ...   
calculated_host_listings_count                  float64
calculated_host_listings_count_entire_homes     float64
calculated_host_listings_count_private_rooms    float64
calculated_host_listings_count_shared_rooms     float64
reviews_per_month                               float64
Length: 75, dtype: object

In [24]:
listings_df.shape

(41535, 75)

In [25]:
# check for missing values
listings_df.isnull().sum()

id                                                 1
listing_url                                        1
scrape_id                                          1
last_scraped                                       0
source                                             0
                                                ... 
calculated_host_listings_count                     4
calculated_host_listings_count_entire_homes        4
calculated_host_listings_count_private_rooms       4
calculated_host_listings_count_shared_rooms        4
reviews_per_month                               9396
Length: 75, dtype: int64

##### Note: A fair amount of missing values in reviews_per_month. The other missing values are insignificant to the entirety of the dataset. Can fill missing values in with 0's and ' '.

In [26]:
listings_df['host_acceptance_rate'].isnull().sum()

12212

In [43]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41535 entries, 0 to 41534
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            41534 non-null  object 
 1   listing_url                                   41534 non-null  object 
 2   scrape_id                                     41534 non-null  object 
 3   last_scraped                                  41535 non-null  object 
 4   source                                        41535 non-null  object 
 5   name                                          41522 non-null  object 
 6   description                                   40748 non-null  object 
 7   neighborhood_overview                         24091 non-null  object 
 8   picture_url                                   41535 non-null  object 
 9   host_id                                       41535 non-null 

##### Check what missing values need to be 0's and which ones should be ' '

In [89]:
float_df = listings_df.select_dtypes(include = float, exclude = object)

listings_df.select_dtypes(include = float, exclude = object).columns

Index(['host_listings_count', 'host_total_listings_count', 'latitude',
       'longitude', 'accommodates', 'bedrooms', 'beds', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'availability_30', 'availability_60', 'availability_90',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')

In [90]:
object_df = listings_df.select_dtypes(include = object, exclude = float)

listings_df.select_dtypes(include = object, exclude = float).columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_verifications', 'host_has_profile_pic',
       'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'property_type', 'room_type',
       'bathrooms', 'bathrooms_text', 'amenities', 'price',
       'maximum_minimum_nights', 'minimum_maximum_nights', 'has_availability',
       'availability_365', 'calendar_last_scraped', 'first_review',
       'last_review', 'license', 'instant_bookable'],
      dtype='object')

In [71]:
# replace missing object values with ' '
object_df.fillna(' ', inplace = True)

# check
object_df.isnull().sum()

id                              0
listing_url                     0
scrape_id                       0
last_scraped                    0
source                          0
name                            0
description                     0
neighborhood_overview           0
picture_url                     0
host_id                         0
host_url                        0
host_name                       0
host_since                      0
host_location                   0
host_about                      0
host_response_time              0
host_response_rate              0
host_acceptance_rate            0
host_is_superhost               0
host_thumbnail_url              0
host_picture_url                0
host_neighbourhood              0
host_verifications              0
host_has_profile_pic            0
host_identity_verified          0
neighbourhood                   0
neighbourhood_cleansed          0
neighbourhood_group_cleansed    0
property_type                   0
room_type     

In [72]:
object_df.shape

(41535, 43)

In [73]:
# replace missing float values with 0
float_df.fillna(0, inplace = True)

# check
float_df.isnull().sum()

host_listings_count                             0
host_total_listings_count                       0
latitude                                        0
longitude                                       0
accommodates                                    0
bedrooms                                        0
beds                                            0
minimum_nights                                  0
maximum_nights                                  0
minimum_minimum_nights                          0
maximum_maximum_nights                          0
minimum_nights_avg_ntm                          0
maximum_nights_avg_ntm                          0
calendar_updated                                0
availability_30                                 0
availability_60                                 0
availability_90                                 0
number_of_reviews                               0
number_of_reviews_ltm                           0
number_of_reviews_l30d                          0


In [74]:
float_df.shape

(41535, 32)

##### Now concatenate object_df and float_df back into the listings_df since there are no more missing values.

In [76]:
listings_df = pd.concat([object_df,float_df], axis = 1) # left/right

# check
listings_df.isnull().sum()

id                                              0
listing_url                                     0
scrape_id                                       0
last_scraped                                    0
source                                          0
                                               ..
calculated_host_listings_count                  0
calculated_host_listings_count_entire_homes     0
calculated_host_listings_count_private_rooms    0
calculated_host_listings_count_shared_rooms     0
reviews_per_month                               0
Length: 75, dtype: int64

##### Save to cleaned csv

In [84]:
# save
listings_df.to_csv('../data/cleaned_listings.csv')

--- 

#### Reviews DF

In [9]:
# check first 5
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,5803.0,1743.0,2009-04-23,9868.0,Pamela,"Artwork, indeed! The walls are covered with dr..."
1,2595.0,17857.0,2009-11-21,50679.0,Jean,Notre séjour de trois nuits.\r<br/>Nous avons ...
2,2595.0,19176.0,2009-12-05,53267.0,Cate,Great experience.
3,2595.0,19760.0,2009-12-10,38960.0,Anita,I've stayed with my friend at the Midtown Cast...
4,2595.0,34320.0,2010-04-09,71130.0,Kai-Uwe,"We've been staying here for about 9 nights, en..."


In [8]:
# check last 5
reviews_df.tail()

,listing_id,id,date,reviewer_id,reviewer_name,comments
1098454,7.654618e+17,7.688403e+17,2022-11-26,79496346.0,Sophie,"It was a nice, clean place to stay. The host w..."
1098455,7.655884e+17,7.695908e+17,2022-11-27,376890617.0,Mark,Lori was absolutely amazing. After two of our...
1098456,7.656736e+17,7.703491e+17,2022-11-28,365712189.0,Jasmine,Great stay Definitely would recommend this hom...
1098457,7.658073e+17,7.681075e+17,2022-11-25,274862833.0,Maria,"Todo muy lindo ,limpio y muy agradable!! El an..."
1098458,7.658073e+17,7.696277e+17,2022-11-27,94373771.0,Victor,"Departamento bien equipado, el anfitrion estuv..."


In [85]:
reviews_df.dtypes

listing_id       float64
id               float64
date              object
reviewer_id      float64
reviewer_name     object
comments          object
dtype: object

In [86]:
reviews_df.shape

(1098459, 6)

In [87]:
# check for missing values
reviews_df.isnull().sum()

listing_id       10087
id               10087
date             10087
reviewer_id      10087
reviewer_name    10091
comments         10336
dtype: int64

##### Do the same thing as listings - separate into float and objects to replace with either ' ' or 0

In [92]:
float_reviews_df = reviews_df.select_dtypes(include = float, exclude = object)

float_reviews_df.select_dtypes(include = float, exclude = object).columns

Index(['listing_id', 'id', 'reviewer_id'], dtype='object')

In [93]:
object_reviews_df = reviews_df.select_dtypes(include = object, exclude = float)

object_reviews_df.select_dtypes(include = object, exclude = float).columns

Index(['date', 'reviewer_name', 'comments'], dtype='object')

In [94]:
# replace missing object values with ' '
object_reviews_df.fillna(' ', inplace = True)

# check
object_reviews_df.isnull().sum()

date             0
reviewer_name    0
comments         0
dtype: int64

In [96]:
object_reviews_df.shape

(1098459, 3)

In [95]:
# replace missing float values with 0
float_reviews_df.fillna(0, inplace = True)

# check
float_reviews_df.isnull().sum()

listing_id     0
id             0
reviewer_id    0
dtype: int64

In [97]:
float_reviews_df.shape

(1098459, 3)

##### Concatenate back together as reviews_df:

In [98]:
reviews_df = pd.concat([object_reviews_df,float_reviews_df], axis = 1) # left/right

# check
reviews_df.isnull().sum()

date             0
reviewer_name    0
comments         0
listing_id       0
id               0
reviewer_id      0
dtype: int64

##### Save to cleaned csv

In [99]:
# save
reviews_df.to_csv('../data/cleaned_reviews.csv')